In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import requests
import numpy as np

from backend.football_backend_client import fetch_upcoming_fixtures

In [2]:
upcoming_fixtures = fetch_upcoming_fixtures()
print(upcoming_fixtures)

      Home_Team_Name     Away_Team_Name
0          Brentford            Arsenal
1            Burnley           West Ham
2            Everton  Manchester United
3             Fulham             Wolves
4              Luton     Crystal Palace
..               ...                ...
255   Crystal Palace        Aston Villa
256        Liverpool             Wolves
257            Luton             Fulham
258  Manchester City           West Ham
259    Sheffield Utd          Tottenham

[260 rows x 2 columns]


In [3]:
df = pd.read_csv('../data/Premier_League_all_match_stats_2015-2023.csv')

In [4]:
label_encoder = LabelEncoder()
df['Home_Team_Name'] = label_encoder.fit_transform(df['Home_Team_Name'])
df['Away_Team_Name'] = label_encoder.transform(df['Away_Team_Name'])

upcoming_fixtures['Home_Team_Name'] = label_encoder.transform(upcoming_fixtures['Home_Team_Name'])
upcoming_fixtures['Away_Team_Name'] = label_encoder.transform(upcoming_fixtures['Away_Team_Name'])

df['Home_passes_%'] = df['Home_passes_%'].str.rstrip('%').astype('float') / 100.0
df['Away_passes_%'] = df['Away_passes_%'].str.rstrip('%').astype('float') / 100.0
df['Home_ball_possession'] = df['Home_ball_possession'].str.rstrip('%').astype('float') / 100.0
df['Away_ball_possession'] = df['Away_ball_possession'].str.rstrip('%').astype('float') / 100.0

df = df.fillna(0)

df = df.drop(['Home_expected_goals', 'Away_expected_goals', 'Away_blocked_shots', 'Away_goalkeeper_saves', 'Away_offsides', 'Away_passes_%', 'Season', 'Match_Date', 'Away_passes_accurate', 'Away_shots_insidebox', 'Away_shots_off_goal', 'Away_shots_on_goal', 'Away_shots_outsidebox', 'Home_blocked_shots', 'Home_goalkeeper_saves', 'Home_offsides', 'Home_passes_%', 'Home_passes_accurate', 'Away_total_passes', 'Home_shots_insidebox', 'Home_shots_off_goal', 'Home_shots_on_goal', 'Home_shots_outsidebox', 'Home_total_passes'], axis=1)

In [5]:
target_columns = ['Home_Goals', 'Away_Goals',
       'Home_total_shots', 'Home_corner_kicks', 'Home_ball_possession',
       'Home_yellow_cards', 'Away_total_shots', 'Away_corner_kicks',
       'Away_ball_possession', 'Away_yellow_cards', 'Away_red_cards',
       'Home_red_cards', 'Home_fouls', 'Away_fouls']

In [6]:
models = {}

for target in target_columns:
    features = df.loc[:, df.columns != target]
    X = features
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    models[target] = model

    mse = mean_squared_error(y_test, model.predict(X_test))
    print(f'Mean Squared Error for {target}: {mse}')

Mean Squared Error for Home_Goals: 1.399967652379675
Mean Squared Error for Away_Goals: 1.052233864229563
Mean Squared Error for Home_total_shots: 15.576261110842918
Mean Squared Error for Home_corner_kicks: 5.939912187364706
Mean Squared Error for Home_ball_possession: 5.435507432457149e-31
Mean Squared Error for Home_yellow_cards: 1.2238640822098719
Mean Squared Error for Away_total_shots: 13.524499708865296
Mean Squared Error for Away_corner_kicks: 5.111503099206632
Mean Squared Error for Away_ball_possession: 5.522727401703309e-31
Mean Squared Error for Away_yellow_cards: 1.4760061419420503
Mean Squared Error for Away_red_cards: 0.0461957373884766
Mean Squared Error for Home_red_cards: 0.06530939661954223
Mean Squared Error for Home_fouls: 13.046643262712568
Mean Squared Error for Away_fouls: 13.592753924413604


In [7]:
for index, row in upcoming_fixtures.iterrows():
    home_team = label_encoder.inverse_transform([row['Home_Team_Name']])[0]
    away_team = label_encoder.inverse_transform([row['Away_Team_Name']])[0]
    teams_involved = f"{home_team} vs {away_team}"
    print(f"\nTeams: {teams_involved}")

    for target in target_columns:
        row_copy = row.copy()
        
        for col, val in zip(target_columns, [0] * len(target_columns)):
            if col != target:
                row_copy[col] = val

        model = models[target]

        reshaped_row = row_copy.values.reshape(1, -1)
        y_pred = model.predict(reshaped_row)

        print(f"{target} Prediction: {y_pred[0]}")
        
    print("\n")


Teams: Brentford vs Arsenal
Home_Goals Prediction: 1.2164532532913042
Away_Goals Prediction: 0.679202778523987
Home_total_shots Prediction: 10.816079830770411
Home_corner_kicks Prediction: 3.742019889782133
Home_ball_possession Prediction: 1.0000000000000022
Home_yellow_cards Prediction: 0.7981062373310494
Away_total_shots Prediction: 8.278065001631877
Away_corner_kicks Prediction: 3.381786317362552
Away_ball_possession Prediction: 0.9999999999999976
Away_yellow_cards Prediction: 1.0584498705565315
Away_red_cards Prediction: 0.004067298087542007
Home_red_cards Prediction: -0.03473991402271359
Home_fouls Prediction: 3.6546493294568267
Away_fouls Prediction: 4.773144190486874



Teams: Burnley vs West Ham
Home_Goals Prediction: 1.2599448580641954
Away_Goals Prediction: 0.6460282016977738
Home_total_shots Prediction: 10.45931875683089
Home_corner_kicks Prediction: 3.739920196780487
Home_ball_possession Prediction: 1.0000000000000027
Home_yellow_cards Prediction: 0.7769462622358888
Away_t

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali

Home_total_shots Prediction: 10.785063397483066
Home_corner_kicks Prediction: 3.7100002790193884
Home_ball_possession Prediction: 1.0000000000000022
Home_yellow_cards Prediction: 0.7909474872498508
Away_total_shots Prediction: 8.304238938712574
Away_corner_kicks Prediction: 3.352361506654915
Away_ball_possession Prediction: 0.9999999999999976
Away_yellow_cards Prediction: 1.0760566923135169
Away_red_cards Prediction: 0.0033048916504348647
Home_red_cards Prediction: -0.03403819277940402
Home_fouls Prediction: 3.61617533996781
Away_fouls Prediction: 4.574145184458977



Teams: Liverpool vs Fulham
Home_Goals Prediction: 1.2181649897691516
Away_Goals Prediction: 0.6664313753846631
Home_total_shots Prediction: 10.746565210721885
Home_corner_kicks Prediction: 3.645001739398747
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.7778424422636301
Away_total_shots Prediction: 8.349331109242312
Away_corner_kicks Prediction: 3.289084356084457
Away_ball_possession P

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali


Teams: Nottingham Forest vs Everton
Home_Goals Prediction: 1.2090983951591274
Away_Goals Prediction: 0.6662136992830652
Home_total_shots Prediction: 10.786425171723062
Home_corner_kicks Prediction: 3.5851301481833304
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.7701644503262389
Away_total_shots Prediction: 8.366112684606902
Away_corner_kicks Prediction: 3.2198808207253546
Away_ball_possession Prediction: 0.9999999999999977
Away_yellow_cards Prediction: 1.1381696075131256
Away_red_cards Prediction: 0.0026310372095156766
Home_red_cards Prediction: -0.029215443123121555
Home_fouls Prediction: 3.7411558721533047
Away_fouls Prediction: 3.759995604226903



Teams: West Ham vs Crystal Palace
Home_Goals Prediction: 1.1978621856802643
Away_Goals Prediction: 0.6656314338118259
Home_total_shots Prediction: 10.834311595894071
Home_corner_kicks Prediction: 3.5082894334913886
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.7

...


Teams: Wolves vs West Ham
Home_Goals Prediction: 1.2306550573348705
Away_Goals Prediction: 0.6411062452076132
Home_total_shots Prediction: 10.56767600962366
Home_corner_kicks Prediction: 3.5108370298473917
Home_ball_possession Prediction: 1.0000000000000027
Home_yellow_cards Prediction: 0.7449928440661825
Away_total_shots Prediction: 8.47508174909859
Away_corner_kicks Prediction: 3.1728839688873682
Away_ball_possession Prediction: 0.9999999999999976
Away_yellow_cards Prediction: 1.1868818615724936
Away_red_cards Prediction: -0.001897182521131296
Home_red_cards Prediction: -0.030090694007493504
Home_fouls Prediction: 3.321861307107595
Away_fouls Prediction: 3.34398205831011



Teams: Bournemouth vs Manchester United
Home_Goals Prediction: 1.2440510269693088
Away_Goals Prediction: 0.6603341127501532
Home_total_shots Prediction: 10.600254480863931
Home_corner_kicks Prediction: 3.7591383143767776
Home_ball_possession Prediction: 1.0000000000000024
Home_yellow_cards Prediction: 0.788377600

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali

Home_ball_possession Prediction: 1.0000000000000022
Home_yellow_cards Prediction: 0.780902575448037
Away_total_shots Prediction: 8.326851778985855
Away_corner_kicks Prediction: 3.26401803678681
Away_ball_possession Prediction: 0.9999999999999977
Away_yellow_cards Prediction: 1.1117593748776473
Away_red_cards Prediction: 0.0037746468651763906
Home_red_cards Prediction: -0.030268024988085913
Home_fouls Prediction: 3.7988668563868293
Away_fouls Prediction: 4.0584941132687495



Teams: Manchester United vs Newcastle
Home_Goals Prediction: 1.2296399931567739
Away_Goals Prediction: 0.655300624408726
Home_total_shots Prediction: 10.640931506409103
Home_corner_kicks Prediction: 3.6243446446597263
Home_ball_possession Prediction: 1.0000000000000027
Home_yellow_cards Prediction: 0.7682297865861129
Away_total_shots Prediction: 8.394356524763635
Away_corner_kicks Prediction: 3.282512100096874
Away_ball_possession Prediction: 0.9999999999999976
Away_yellow_cards Prediction: 1.1264317853397705
Away_

C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\x_fernare1\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have vali

Home_fouls Prediction: 3.714260039703279
Away_fouls Prediction: 4.497234109901019



Teams: Liverpool vs Wolves
Home_Goals Prediction: 1.250569726289089
Away_Goals Prediction: 0.6431468681574187
Home_total_shots Prediction: 10.48768373277331
Home_corner_kicks Prediction: 3.6555542384454363
Home_ball_possession Prediction: 1.0000000000000029
Home_yellow_cards Prediction: 0.764505435325687
Away_total_shots Prediction: 8.429143849190545
Away_corner_kicks Prediction: 3.337787176791484
Away_ball_possession Prediction: 0.9999999999999974
Away_yellow_cards Prediction: 1.1190956686056617
Away_red_cards Prediction: -0.002590260620244614
Home_red_cards Prediction: -0.03701734028052801
Home_fouls Prediction: 3.0007113754511634
Away_fouls Prediction: 4.311953787658099



Teams: Luton vs Fulham
Home_Goals Prediction: 1.2170801823347321
Away_Goals Prediction: 0.6662490806998423
Home_total_shots Prediction: 10.750578442306804
Home_corner_kicks Prediction: 3.636517177660484
Home_ball_possession Predic